# NYCU Machine Learning 2024 : HW4 SVM

In [38]:
import numpy as np
import cvxpy as cp
import pandas as pd

from pathlib import Path
from rich import print
from dataclasses import dataclass
from typing import Callable

In [39]:
LABEL = ["Setosa" , "Versicolor" , "Virginica" ]

COLOR_1 = dict(zip(LABEL, ["red" , "green" , "blue"]))
COLOR_2 = dict(zip(LABEL, ["pink" , "yellow" , "orange"]))
COLOR_3 = dict(zip(LABEL, ["brown", "lightgreen", "navy", "magenta"]))
COLOR_4 = dict(zip(LABEL, ["teal", "gold", "violet", "coral"]))

COLOR_SELECT = {
    "before": {
        "train":COLOR_1,
        "test":COLOR_2,
    },
    "after": {
        "train":COLOR_3,
        "test":COLOR_4,
    }
}

COLUMN_NAME = ["Sepal length", "Sepal width" , "Petal length" , "Petal width" , "Label"]
TRAIN_DATA_SIZE = 25
ASSETS = "./assets"

In [40]:
assets_folder = Path(ASSETS)
assets_folder.mkdir(parents=True, exist_ok=True) 

In [41]:
def load_iris_file(with_name:bool=False)->pd.DataFrame:
    df = pd.read_fwf("./iris.txt")
    
    df_new = pd.DataFrame({k:[v] for k ,v in zip(COLUMN_NAME , df.columns)},dtype=float)
    df.columns = COLUMN_NAME
    df_new = pd.concat([df_new, df], axis=0).reset_index().drop(columns=["index"])
    
    if not with_name:
        return df_new
    
    df_with_name = df_new.copy()
    
    df_with_name["Label"] = df_with_name["Label"].apply(lambda x : LABEL[int(x)-1])
    
    return df_with_name

In [42]:
df = load_iris_file(with_name=True)
df

,Sepal length,Sepal width,Petal length,Petal width,Label
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica


In [43]:
POSITIVE_CLASS ,NEGATIVE_CLASS= "Versicolor" , "Virginica"

In [44]:
df_need = df[["Petal length","Petal width", "Label"]]
df_need

,Petal length,Petal width,Label
0,1.4,0.2,Setosa
1,1.4,0.2,Setosa
2,1.3,0.2,Setosa
3,1.5,0.2,Setosa
4,1.4,0.2,Setosa
...,...,...,...
145,5.2,2.3,Virginica
146,5.0,1.9,Virginica
147,5.2,2.0,Virginica
148,5.4,2.3,Virginica


In [45]:
df_need_class = df_need[(df_need["Label"] == POSITIVE_CLASS) | (df_need["Label"] == NEGATIVE_CLASS)]
df_need_class = df_need_class.reset_index().drop(columns=["index"])
df_need_class

,Petal length,Petal width,Label
0,4.7,1.4,Versicolor
1,4.5,1.5,Versicolor
2,4.9,1.5,Versicolor
3,4.0,1.3,Versicolor
4,4.6,1.5,Versicolor
...,...,...,...
95,5.2,2.3,Virginica
96,5.0,1.9,Virginica
97,5.2,2.0,Virginica
98,5.4,2.3,Virginica


In [46]:
label_to_index = {POSITIVE_CLASS : 1 , NEGATIVE_CLASS : -1}
index_to_label = {1 : POSITIVE_CLASS , -1 : NEGATIVE_CLASS}


label_to_index_np = np.vectorize(label_to_index.get)
index_to_label_np = np.vectorize(index_to_label.get)

In [47]:
dummy_data_x , dummy_data_y = df_need_class.drop(columns=["Label"]).to_numpy() , df_need_class["Label"].to_numpy() 

In [48]:
dummy_data_x.shape

(100, 2)

In [49]:
dummy_data_y = label_to_index_np(dummy_data_y)
dummy_data_y

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

## Model

In [50]:
def rbf(sigma:float) -> Callable[[np.ndarray, np.ndarray], np.ndarray]:
    def run(x_1:np.ndarray, x_2:np.ndarray)->np.ndarray:
        assert x_1.shape[0] == x_2.shape[0], "输入的两个向量必须有相同的特征数"
        over = 1/ (2* np.square(sigma))
        dis = -np.linalg.norm(x_1 - x_2)**2 
        
        return np.exp(dis* over)
    
    return run

def poly(p:int)-> Callable[[np.ndarray, np.ndarray], np.ndarray]:
    def run(x_1:np.ndarray, x_2:np.ndarray)->np.ndarray:
        assert x_1.shape[0] == x_2.shape[0], "输入的两个向量必须有相同的特征数"
        
        return (x_1.T @ x_2)**p
    
    return run

def linear() -> Callable[[np.ndarray, np.ndarray], np.ndarray]:
    
    def run(x_1:np.ndarray, x_2:np.ndarray)->np.ndarray:
        assert x_1.shape[0] == x_2.shape[0], "输入的两个向量必须有相同的特征数"
        
        return x_1.T @ x_2
    
    return run

class Kernel:
    kernel_dict = {
        "linear":linear,
        "rbf": rbf,
        "poly": poly,
    }
        
    @staticmethod
    def get_kernel(name:str, config:dict) -> Callable[[np.ndarray, np.ndarray], np.ndarray]:
        if name not in Kernel.kernel_dict:
            raise NotImplementedError(f"{name} kernel not implemented. Available kernels: {list(Kernel.kernel_dict.keys())}")
        
        func = Kernel.kernel_dict[name]
        
        return func(**config) 

In [51]:
kernel = Kernel.get_kernel("rbf", {"sigma":1})

In [52]:
class SupportVectorMachine:
    def __init__(self, C:int, kernel_name:str="linear", kernel_arg:dict=dict(),threshold:float=1e-20):
        self._c = C
        self._threshold = threshold
        # like ("ay": ... , "x": ...,)
        self._a_x_y :np.ndarray = None
        self._b :float = None
        self._kernel  = Kernel.get_kernel(kernel_name, kernel_arg)
        return
    
    def _build_k_matrix(self, x:np.ndarray)->np.ndarray:
        size = x.shape[0]
        return np.array([[self._kernel(x[i], x[j])  for j in range(size)] for i in range(size)])
    
    def _find_alpha(self, K:np.ndarray, x:np.ndarray , y:np.ndarray)-> tuple[np.ndarray,np.ndarray]:
        size = x.shape[0] 
        alpha = cp.Variable(size)
        
        big_k = np.diag(y) @ K @ np.diag(y)
        big_k = cp.psd_wrap(big_k)
        
        objective = cp.Minimize((1/2) * cp.quad_form(alpha, big_k) -cp.sum(alpha))
        constraints = [
            cp.sum(cp.multiply(alpha, y)) == 0,  
            alpha >= 0, 
            alpha <= self._c
        ]
        problem = cp.Problem(objective, constraints) 
        problem.solve()
        
        alpha = alpha.value.reshape(-1,1)
        
        # filter the important one 
        support_vectors = np.where((self._c >= alpha) & (alpha > self._threshold))[0]
        
        return alpha , support_vectors
    
    def _find_bias(self,alpha:np.ndarray,support_vectors:np.ndarray, y: np.ndarray, K: np.ndarray):
        res = [y[i] - np.sum([alpha[j]*y[j]*K[i][j] for j in support_vectors]) for i in support_vectors]
        
        return np.mean(res)
    
    def train(self, x:np.ndarray, y:np.ndarray): # [batch, feature]   [batch, 1]
        # get the a
        K = self._build_k_matrix(x) 
        alpha , support_vector = self._find_alpha(K,x,y)
        # find the best b
        self._b = self._find_bias(alpha,support_vector,y,K)
        
        print(self._b)
        
        table = np.hstack((alpha * y.reshape(-1,1), x ))
        
        self._a_x_y = table[support_vector]
        print(self._a_x_y)
        
        return 
    
    def cal_one_item(self, ay:np.ndarray, x_kernel:np.ndarray, x_item: np.ndarray)->np.ndarray:
        # x [1, feature]
        # x_kernel [a, feature]
        
        res = np.sum(ay * self._kernel(x_kernel.T, x_item.reshape(-1,1))) + self._b
        
        return res
    
    def __call__(self, x : np.ndarray)->np.ndarray:
        # x [batch, feature]
        ay, x_kernel = self._a_x_y[:, 0].reshape(-1,1) ,self._a_x_y[:, 1:]
        
        result = [self.cal_one_item(ay, x_kernel=x_kernel, x_item=x_item) for x_item in x]
        
        res = np.hstack(result) 
        return res
    

In [53]:
model = SupportVectorMachine(C =1 , kernel_name="linear" )

In [54]:
model.train(x=dummy_data_x[:2], y=dummy_data_y[:2])

0.9999880549772212

[[5.30889901e-07 4.50000000e+00 1.50000000e+00]]

In [55]:
model(dummy_data_x)

array([1.0000004 , 1.        , 1.00000096, 0.99999865, 1.00000024,
       0.99999984, 1.00000056, 0.99999674, 1.00000008, 0.99999849,
       0.99999721, 0.99999928, 0.99999841, 1.0000004 , 0.99999769,
       0.99999968, 1.        , 0.99999865, 1.        , 0.99999825,
       1.00000096, 0.99999865, 1.00000096, 1.00000024, 0.99999936,
       0.99999968, 1.00000064, 1.00000135, 1.        , 0.99999721,
       0.99999801, 0.99999769, 0.99999833, 1.00000151, 1.        ,
       1.00000008, 1.00000048, 0.9999996 , 0.99999889, 0.99999865,
       0.99999952, 1.00000016, 0.99999857, 0.99999674, 0.99999912,
       0.99999904, 0.99999912, 0.99999936, 0.9999961 , 0.99999889,
       1.00000438, 1.00000175, 1.00000382, 1.00000287, 1.00000366,
       1.00000549, 1.00000016, 1.00000454, 1.00000334, 1.00000462,
       1.00000183, 1.00000223, 1.00000287, 1.00000159, 1.00000215,
       1.00000255, 1.00000263, 1.00000581, 1.00000637, 1.00000119,
       1.0000035 , 1.00000135, 1.00000565, 1.00000119, 1.00000